In [33]:
#01 RGB
import cv2

def BGR2RGB(img):
    r = img[:, :, 2].copy()
    g = img[:, :, 1].copy()
    b = img[:, :, 0].copy()

    img[:,:,0] = r
    img[:,:,1] = g
    img[:,:,2] = b

    return img

img = cv2.imread("imori.jpg")
img = BGR2RGB(img)

# Save result
cv2.imwrite("myanswers_py/out_01.jpg", img)
# cv2.imshow("result", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [34]:
#02 GrayScale
import cv2
import numpy as np

# astype 类型转换
def grayscale(img):
    b = img[:,:,0].copy()
    g = img[:,:,1].copy()
    r = img[:,:,2].copy()

    y = 0.2126*r + 0.7152*g + 0.0722*b #浮点运算，读取Img时转换成浮点类型

    y = y.astype(np.uint8)

    return y

img = cv2.imread("imori.jpg")
gray = grayscale(img)
cv2.imwrite("myanswers_py/out_02.jpg",gray)


True

In [35]:
#03 Binarization
import cv2
import numpy as np

# 二值化
def binarization(img,th=128):
    img[img < th] = 0
    img[img >= th ] = 255
    return img 

img = cv2.imread("myanswers_py/out_02.jpg")
t = binarization(img,th=128)
cv2.imwrite("myanswers_py/out_03.jpg",t)


True

In [36]:
#04 Otsu's Method
import cv2
import numpy as np

def otsu_binarization(gray_img):
    #最大类间方差算法
    max_t = 0
    max_sigma = 0
    H,W,_ = gray_img.shape

    for _t in range(1, 255):
        c0 = gray_img[np.where(gray_img < _t)]
        c1 = gray_img[np.where(gray_img >= _t)]

        w0 = len(c0)/(H*W)
        w1 = len(c1)/(H*W)

        m0 = np.mean(c0) if len(c0) > 0 else 0
        m1 = np.mean(c1) if len(c1) > 0 else 0

        sigma = w0*w1 *((m0-m1)**2)
        if sigma > max_sigma:
            max_sigma = sigma
            max_t = _t

    print("threshold >>", max_t)
    th = max_t
    gray_img[gray_img < th] = 0
    gray_img[gray_img >= th] = 255

    return gray_img

img = cv2.imread("myanswers_py/out_02.jpg")
out = otsu_binarization(img)
cv2.imwrite("myanswers_py/out_04.jpg",out)



threshold >> 127


True

In [37]:
#05 HSV变换
import cv2 
import numpy as np

def BGR2HSV(_img):
    #RGB的取值范围为0到1
    img = _img.copy() / 255.

    hsv = np.zeros_like(img,dtype = np.float32)

    #取RGB中最大的,min_arg找到最小的那一维
    max_v = np.max(img, axis=2).copy()
    min_v = np.min(img, axis=2).copy()
    min_arg = np.argmin(img, axis=2)

    #H
    hsv[...,0][np.where(max_v == min_v)] = 0
    ## Min=B
    #np.where返回满足条件的坐标
    ind = np.where(min_arg == 0)
    hsv[...,0][ind] = 60 * (img[...,1][ind]-img[...,2][ind]) / (max_v[ind]-min_v[ind]) + 60
    ## Min=R
    ind = np.where(min_arg == 2)
    hsv[...,0][ind] = 60 * (img[...,0][ind]-img[...,1][ind]) / (max_v[ind]-min_v[ind]) + 180
    ## Min=G
    ind = np.where(min_arg == 1)
    hsv[...,0][ind] = 60 * (img[...,2][ind]-img[...,0][ind]) / (max_v[ind]-min_v[ind]) + 300

    #S
    hsv[...,1] = max_v.copy() - min_v.copy()

    #V
    hsv[...,2] = max_v.copy()

    return hsv

def HSV2BGR(_img,hsv):
    img =_img.copy() / 255.

    max_v = np.max(img, axis=2).copy()
    min_v = np.min(img, axis =2).copy()

    out = np.zeros_like(img)

    H = hsv[...,0]
    S = hsv[...,1]
    V = hsv[...,2]
   
    C = S
    H_ = H / 60.
    X = C * (1 - np.abs( H_ % 2 - 1))
    Z = np.zeros_like(H)

    vals = [[Z,X,C], [Z,C,X], [X,C,Z], [C,X,Z], [C,Z,X], [X,Z,C]]

    for i in range(6):
        ind = np.where((i <= H_) & (H_ < (i+1)))
        out[...,0][ind] = (V - C)[ind] + vals[i][0][ind]
        out[...,1][ind] = (V - C)[ind] + vals[i][1][ind]
        out[...,2][ind] = (V - C)[ind] + vals[i][2][ind]
    out[np.where(max_v == min_v)] = 0
    out = np.clip(out, 0 , 1)
    out = (out * 255).astype(np.uint8)

    return out

img = cv2.imread("imori.jpg").astype(np.float32)

# RGB > HSV
hsv = BGR2HSV(img)

# 将色相反转，再用RGB
# Transpose Hue
hsv[..., 0] = (hsv[..., 0] + 180) % 360

# HSV > RGB
out = HSV2BGR(img, hsv)

# Save result
cv2.imwrite("myanswers_py/out_05.jpg", out)

True

In [38]:
#06 减色处理
#色彩量化
import cv2
import numpy as np

def decrease_color(img):
    out = img.copy()
    out = out//64 * 64 + 32
    
    return out

img = cv2.imread("imori.jpg").astype(np.uint8)
out = decrease_color(img)
cv2.imwrite("myanswers_py/out_06.jpg",out)

True

In [39]:
#07 平均池化
#按照固定大小分割，网格内取像素平均值
import cv2
import numpy as np

def average_pooling(img, G=8):
    out = img.copy()
    H,W,C = img.shape

    Nh = int(H / G)
    Nw = int(W / G)

    for y in range(Nh):
        for x in range(Nw):
            for c in range(C):
                out[G*y:G*(y+1) , G*x:G*(x+1) , c] = np.mean(out[G*y:G*(y+1) , G*x:G*(x+1) , c]).astype(int)

    return out

img = cv2.imread("imori.jpg")
out = average_pooling(img)

cv2.imwrite("myanswers_py/out_07.jpg",out)

True

In [40]:
#08 最大池化
import cv2
import numpy as np

def max_pooling(img, G=8):
    out = img.copy()
    H,W,C = img.shape

    Nh = int(H / G)
    Nw = int(W / G)

    for y in range(Nh):
        for x in range(Nw):
            for c in range(C):
                out[G*y:G*(y+1) , G*x:G*(x+1) , c] = np.max(out[G*y:G*(y+1) , G*x:G*(x+1) , c]).astype(int)

    return out

img = cv2.imread("imori.jpg")
out = max_pooling(img)

cv2.imwrite("myanswers_py/out_08.jpg",out)

True

In [41]:
#09 高斯滤波
import cv2
import numpy as np

def gaussian_filter(img, K_size=3, sigma=1.3):
    # 确保是三通道的图片输入
    # np.expand_dims在第axis维，加一个维度，原有的维度推到右边去
    if len(img.shape) == 3 :
        H, W, C = img.shape
    else:
        img = np.expand_dims(img, axis=-1)
        H, W, C = img.shape

    ## Zero padding
    #H W 两侧都补零
    pad = K_size // 2
    out = np.zeros((H + pad*2, W + pad*2, C), dtype=np.float)
    out[pad: pad + H, pad: pad + W] = img.copy().astype(np.float32)

    ## 计算卷积核Kernel
    # 根据公式，并进行归一化操作
    # zeros 和 zeros_like的区别
    K = np.zeros((K_size, K_size) , dtype=np.float32)
    for y in range(-pad , -pad + K_size):
        for x in range(-pad , -pad + K_size):
                K [x + pad, y + pad] = np.exp( -(x**2 + y**2) / (2*sigma*sigma))
    K /= (2 * np.pi * sigma * sigma)
    K /= K.sum()

    ## 进行滤波；卷积操作
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.sum(K * tmp[y: y + K_size, x: x + K_size, c ])

    ## 格式处理
    out = np.clip(out, 0, 255)
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)

    return out

img = cv2.imread("imori_noise.jpg")

out = gaussian_filter(img, K_size=3, sigma=1.3)

cv2.imwrite("myanswers_py/out_09.jpg",out)


/home/zuochenyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


True

np.zeros_like & np.zeros
Modern OS allocate memory virtually, ie., memory is given to a process only when it is first used. zeros obtains memory from the operating system so that the OS zeroes it when it is first used. zeros_like on the other hand fills the alloced memory with zeros by itself. Both ways require about same amount of work --- it's just that with zeros_like the zeroing is done upfront, whereas zeros ends up doing it on the fly.
之前报错zeros_like维度不对？
zeros_like 输出与输入数组一样大小的数组

In [43]:
#10 中值滤波
import cv2
import numpy as np

def median_filter(img, K_size=3):
    # 确保是三通道的图片输入
    # np.expand_dims在第axis维，加一个维度，原有的维度推到右边去
    if len(img.shape) == 3 :
        H, W, C = img.shape
    else:
        img = np.expand_dims(img, axis=-1)
        H, W, C = img.shape

    ## Zero padding
    #H W 两侧都补零
    pad = K_size // 2
    out = np.zeros((H + pad*2, W + pad*2, C), dtype=np.float)
    out[pad: pad + H, pad: pad + W] = img.copy().astype(np.float32)

    ## 进行滤波；卷积操作
    tmp = out.copy()
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad+y, pad+x, c] = np.median(tmp[y: y + K_size, x: x + K_size, c ])

    ## 格式处理
    # out = np.clip(out, 0, 255) # 没有进行卷积操作，不用Clip
    out = out[pad: pad + H, pad: pad + W].astype(np.uint8)

    return out

img = cv2.imread("imori_noise.jpg")

out = median_filter(img, K_size=3)

cv2.imwrite("myanswers_py/out_10.jpg",out)


/home/zuochenyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


True